In [4]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
from sklearn.kernel_ridge import KernelRidge
import numpy.random as rd
from tqdm import tqdm

In [5]:
def euclidean_kernel(x, xi):
    return np.exp(-np.linalg.norm(x - xi)**2)

def compute_kernel_matrix(X, Y):
    l = X.shape[0]
    r = Y.shape[0]
    K = np.zeros((l, r))
    for i in range(l):
        for j in range(r):
            K[i, j] = euclidean_kernel(X[i], Y[j])
    return K

def compute_alpha_star(Kmm, Knm, y, sigma_squared, nu):
    m = Kmm.shape[0]
    A = sigma_squared * Kmm + np.dot(Knm.T, Knm) + nu * np.eye(m)
    b = np.dot(Knm.T, y)
    alpha_star = np.linalg.solve(A, b)
    return alpha_star
    
def local_objective_function(alpha, sigma_squared, K_mm, y_local, K_im, nu, a):
    term1 = (sigma_squared / a) * (1 / 2) * np.dot(np.dot(alpha.T, K_mm), alpha)
    term2 = (1 / 2) * np.sum((y_local - K_im @ alpha)**2)
    term3 = (nu / (2 * a)) * np.linalg.norm(alpha)**2
    return term1 + term2 + term3

def compute_local_gradient(alpha, sigma_squared, K_mm, y_local, K_im, nu, a):
    # print("alpha local ", alpha)
    # print("K_mm @ alpha",K_mm @ alpha)
    grad = sigma_squared * K_mm @ alpha / a
    # print("K_im.T.shape:",K_im.T.shape)
    # print("(K_im @ alpha - y_local).shape:",(K_im @ alpha - y_local).shape)
    # print("K_im.T @ (K_im @ alpha - y_local) ",K_im.T @ (K_im @ alpha - y_local))
    grad += K_im.T @ (K_im @ alpha - y_local)
    # print("grad.shape:",grad.shape)
    # print("alpha.shape:",alpha.shape)
    # print("(nu / a) * alpha",(nu / a) * alpha)
    grad += (nu / a) * alpha
    return grad

def Nystrom_approx_f(alpha,X_selected,X):
    K1m = compute_kernel_matrix(X,X_selected)
    # print("K1m",K1m.shape)
    # print("alpha.shape",alpha.shape)
    return K1m @ alpha

In [6]:
import networkx as nx

def generate_line_graph(a):
    """
    Generate a line graph where each agent communicates only with its immediate neighbors.

    Parameters:
    a (int): Number of agents.

    Returns:
    np.ndarray: Adjacency matrix representing the line graph.
    """
    line_graph = np.zeros((a, a))
    for i in range(a - 1):
        line_graph[i, i + 1] = 1
        line_graph[i + 1, i] = 1
    np.fill_diagonal(line_graph, 1)
    return line_graph

def generate_small_world_graph(a, p=0.1):
    """
    Generate a small-world graph with a specified number of agents and probability of rewiring.

    Parameters:
    a (int): Number of agents.
    p (float): Probability of rewiring.

    Returns:
    np.ndarray: Adjacency matrix representing the small-world graph.
    """
    small_world_graph = nx.watts_strogatz_graph(a, 2, p)
    small_world_graph=nx.to_numpy_array(small_world_graph)
    np.fill_diagonal(small_world_graph, 1)
    return small_world_graph

def generate_fully_connected_graph(a):
    """
    Generate a fully connected graph where each agent communicates with every other agent directly.

    Parameters:
    a (int): Number of agents.

    Returns:
    np.ndarray: Adjacency matrix representing the fully connected graph.
    """
    return np.ones((a, a)) /a

def generate_cycle_graph(a):
    """
    Generate a cycle graph where each agent communicates with its two adjacent agents forming a cycle.

    Parameters:
    a (int): Number of agents.

    Returns:
    np.ndarray: Adjacency matrix representing the cycle graph.
    """
    cycle_graph = np.zeros((a, a))
    for i in range(a):
        cycle_graph[i, (i - 1) % a] = 1  # Left neighbor
        cycle_graph[i, (i + 1) % a] = 1  # Right neighbor
    np.fill_diagonal(cycle_graph,1)
    return cycle_graph

def renormalize_graph(graph):
    """
    Renormalize the given graph to make it doubly stochastic.

    Parameters:
    graph (np.ndarray): Adjacency matrix representing the graph.

    Returns:
    np.ndarray: Doubly stochastic matrix obtained by renormalizing the input graph.
    """
    row_sums = np.sum(graph, axis=1)
    #print("row_sums:",row_sums)
    graph = graph / row_sums[:, np.newaxis]
    # print("row normalized graph",graph)
    col_sums = np.sum(graph, axis=0)
    #print("col_sums:",col_sums)
    graph = graph / col_sums[np.newaxis,:]
    return graph

In [7]:
def decentralized_gradient_descent(X, X_selected, y, sigma_squared, nu, a, num_iterations, step_size, alpha_star, W):
    m = X_selected.shape[0]
    n = X.shape[0]
    agents_data_indices = np.array_split(np.random.permutation(n), a)
    # each agent should have n/a=100/5=20 points
    #print("agents_data_indices:",agents_data_indices)
    alpha = np.ones((m * a, 1))  # Initialize local variables for each agent
    all_optimality_gaps = [[] for _ in range(a)]  # Store optimality gaps for each agent
    all_alpha = []  # Store alpha at each iteration
    
    # Create mixing matrix from communication matrix
    mixing_matrix = np.kron(W, np.eye(m))  # Mixing matrix
    
    # Compute kernel matrices
    K_mm = compute_kernel_matrix(X_selected, X_selected)
    #print("K_mm",K_mm)
    #Knm = compute_kernel_matrix(X, X_selected)
    #print("Knm ",Knm)
    
    for iter in tqdm(range(num_iterations)):
        #print("iter:",iter)
        
        # Append current alpha to the list
        all_alpha.append(alpha)
        
        all_gradients = np.zeros((m * a, 1))
        
        # Compute gradients for all agents
        for agent_idx, data_indices in enumerate(agents_data_indices):
            # Select data and compute local variables
            X_local = X[data_indices]
            y_local = y[data_indices].reshape(-1, 1)
            K_im = compute_kernel_matrix(X_local, X_selected)
            
            # Compute local gradient on local copy of alpha
            grad_local = compute_local_gradient(alpha[m * agent_idx: m * (agent_idx + 1)], sigma_squared, K_mm, y_local, K_im, nu, a)
            all_gradients[m * agent_idx: m * (agent_idx + 1)] = grad_local
        
        # Update alpha simultaneously for all agents

        alpha = mixing_matrix @ alpha - step_size * all_gradients
        
        # Compute and store optimality gaps for each agent
        for agent_idx in range(a):
            alpha_agent = alpha[m * agent_idx: m * (agent_idx + 1)].reshape(-1, 1)
            optimality_gap = np.linalg.norm(alpha_agent - alpha_star.reshape(-1, 1))
            all_optimality_gaps[agent_idx].append(optimality_gap)
        
    return all_optimality_gaps, all_alpha


In [8]:
def decentralized_gradient_tracking(X, X_selected, y, sigma_squared, nu, a, num_iterations, step_size, alpha_star, W):
    m = X_selected.shape[0]
    n = X.shape[0]
    agents_data_indices = np.array_split(np.random.permutation(n), a)
    # each agent should have n/a=100/5=20 points
    #print("agents_data_indices:",agents_data_indices)
    alpha = np.ones((m * a, 1))  # Initialize local variables for each agent
    all_optimality_gaps = [[] for _ in range(a)]  # Store optimality gaps for each agent
    all_alpha = []  # Store alpha at each iteration
    gradient_tracker = np.zeros((m * a, 1))  # Initialize gradient tracker
    
    # Create mixing matrix from communication matrix
    mixing_matrix = np.kron(W, np.eye(m))  # Mixing matrix
    
    # Compute kernel matrices
    K_mm = compute_kernel_matrix(X_selected, X_selected)
    #print("K_mm",K_mm)
    Knm = compute_kernel_matrix(X, X_selected)
    #print("Knm ",Knm)
    grad0 = np.zeros((m * a, 1))
    grad1 =  np.zeros((m * a, 1)) #store the two last gradients for all agents
    for iter in tqdm(range(num_iterations)):
        #print("iter:",iter)
        
        # Append current alpha to the list
        all_alpha.append(alpha)
        
        all_gradients = np.zeros((m * a, 1))
        
        # Compute gradients for all agents
        for agent_idx, data_indices in enumerate(agents_data_indices):
            # Select data and compute local variables
            X_local = X[data_indices]
            y_local = y[data_indices].reshape(-1, 1)
            K_im = compute_kernel_matrix(X_local, X_selected)
            
            # Compute local gradient on local copy of alpha
            grad_local = compute_local_gradient(alpha[m * agent_idx: m * (agent_idx + 1)], sigma_squared, K_mm, y_local, K_im, nu, a)
            all_gradients[m * agent_idx: m * (agent_idx + 1)] = grad_local
        grad0 = grad1
        grad1 = all_gradients
         # Gradient tracking update
        gradient_tracker = mixing_matrix @ gradient_tracker + grad1 - grad0
        # Update alpha using the gradient tracker
        alpha = mixing_matrix @ alpha - step_size * gradient_tracker
        
        # Compute and store optimality gaps for each agent
        for agent_idx in range(a):
            alpha_agent = alpha[m * agent_idx: m * (agent_idx + 1)].reshape(-1, 1)
            optimality_gap = np.linalg.norm(alpha_agent - alpha_star.reshape(-1, 1))
            all_optimality_gaps[agent_idx].append(optimality_gap)
        
    return all_optimality_gaps, all_alpha


In [9]:
def step(b, Kmm, Knm, Knmy, nu, sigma_squared):
    m = Kmm.shape[0]
    A = (1/5)*sigma_squared * Kmm + Knm + (nu/5) * np.eye(m)
    b_ = Knmy - b
    alpha_star_i = np.linalg.solve(A, b_)
    return alpha_star_i

def create_lamb(a, m):
    lamb = {}
    for i in range(a):
        for j in range(a):
            if i > j:
                #lamb[(i,j)] = np.random.rand(m).reshape(m,1)
                lamb[(i,j)] = np.zeros(m).reshape(m,1)
    return(lamb)

def term_sup(lam, W, m, i):
    a = W.shape[0]
    res = np.zeros((m,1))
    for j in range(a):
        if W[i,j] > 0:
            if i > j:
                res += lam[i,j]
            elif j > i:
                res -=  lam[j,i]
    return(res)

def dual_decomposition(X, X_selected, y, sigma_squared, nu, a, num_iterations, step_size, alpha_star, W):
    m = X_selected.shape[0]
    n = X.shape[0]
    agents_data_indices = np.array_split(np.random.permutation(n), a)
    # each agent should have n/a=100/5=20 points
    #print("agents_data_indices:",agents_data_indices)
    alpha = [np.zeros((m, 1)) for _ in range(a)]  # Initialize local variables for each agent
    all_optimality_gaps = [[] for _ in range(a)]  # Store optimality gaps for each agent
    all_alpha = [[] for _ in range(a)]  # Store alpha at each iteration
    lamb = create_lamb(a, m)
    K_mm = compute_kernel_matrix(X_selected, X_selected)
    #Knm = compute_kernel_matrix(X, X_selected)
    for k in tqdm(range(num_iterations)):
        for agent_idx, data_indices in enumerate(agents_data_indices):
            # Select data and compute local variables
            X_local = X[data_indices]
            y_local = y[data_indices].reshape(-1, 1)
            K_im = np.zeros((m,m))
            K_imy = np.zeros((m,1))
            for j in range(len(X_local)):
                K_jm = compute_kernel_matrix(np.array([X_local[j]]), X_selected)
                K_jmy = y_local[j]*K_jm.T
                KK_im = np.dot(K_jm.T, K_jm)
                K_im += KK_im
                K_imy += K_jmy
            b = term_sup(lamb, W, m, agent_idx)
            #print(b)
            sol = step(b, K_mm, K_im, K_imy, nu, sigma_squared)
            alpha[agent_idx] = sol
        for i in range(a):
            for j in range(i):
                lamb[i,j] += step_size[k]*(alpha[i] - alpha[j])
        for agent_idx in range(a):
            alpha_agent = alpha[agent_idx]
            optimality_gap = np.linalg.norm(alpha_agent - alpha_star.reshape(-1, 1))
            all_optimality_gaps[agent_idx].append(optimality_gap)
            all_alpha[agent_idx].append(alpha_agent)

    return all_optimality_gaps, all_alpha



In [10]:
def create_lamb_gam(a, m):
    lamb = {}
    gam = {}
    for i in range(a):
        for j in range(a):
            lamb[(i,j)] = np.zeros(m).reshape(m,1)
            gam[(i,j)] =  np.random.rand(m).reshape(m,1)
    return(lamb, gam)

def step2(b,c, Kmm, Knm,Knmy, nu, sigma_squared):
    m = Kmm.shape[0]
    A = (1/5)*sigma_squared * Kmm + Knm + ((nu/5) +c) * np.eye(m)
    b_ = Knmy - b
    alpha_star_i = np.linalg.solve(A, b_)
    return alpha_star_i

def term_sup2(gam, lamb, beta, W, i):
    c = 0
    b = 0
    a = W.shape[0]
    for j in range(a):
        if W[i,j] > 0:
            c += beta
            b += lamb[i,j] - beta*gam[i,j]
    return b, c

def ADMM(X, X_selected, y, sigma_squared, nu, a, num_iterations, beta, alpha_star, W):
    m = X_selected.shape[0]
    n = X.shape[0]
    agents_data_indices = np.array_split(np.random.permutation(n), a)
    # each agent should have n/a=100/5=20 points
    #print("agents_data_indices:",agents_data_indices)
    alpha = [np.ones((m, 1)) for _ in range(a)]  # Initialize local variables for each agent
    all_optimality_gaps = [[] for _ in range(a)]  # Store optimality gaps for each agent
    all_alpha = [[] for _ in range(a)]  # Store alpha at each iteration
    lamb, gam = create_lamb_gam(a, m)
    K_mm = compute_kernel_matrix(X_selected, X_selected)
    #Knm = compute_kernel_matrix(X, X_selected)
    for _ in tqdm(range(num_iterations)):
        for agent_idx, data_indices in enumerate(agents_data_indices):
            # Select data and compute local variables
            X_local = X[data_indices]
            y_local = y[data_indices].reshape(-1, 1)
            K_im = np.zeros((m,m))
            K_imy = np.zeros((m,1))
            for j in range(len(X_local)):
                K_jm = compute_kernel_matrix(np.array([X_local[j]]), X_selected)
                K_jmy = y_local[j]*K_jm.T
                KK_im = np.dot(K_jm.T, K_jm)
                K_im += KK_im
                K_imy += K_jmy
            b, c = term_sup2(gam, lamb,beta, W, agent_idx)
            sol = step2(b,c, K_mm, K_im, K_imy, nu, sigma_squared)
            alpha[agent_idx] = sol
        for i in range(a):
            for j in range(i):
                gam[i,j] = (1/2)*(alpha[i] + alpha[j])
                lamb[i,j] += beta*(alpha[i] - gam[i,j])
        for agent_idx in range(a):
            alpha_agent = alpha[agent_idx]
            optimality_gap = np.linalg.norm(alpha_agent - alpha_star.reshape(-1, 1))
            all_optimality_gaps[agent_idx].append(optimality_gap)
            all_alpha[agent_idx].append(alpha_agent)

    return all_optimality_gaps, all_alpha